In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-20 17:08:15--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 17:08:16 (31.3 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [15]:
import os
import openai 
import minsearch 
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
with open("documents.json") as f:
    file = json.load(f)

In [3]:
documents =[]

for course_dict in file:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
    )

In [6]:
index.fit(documents)

In [7]:
q = "this course alreaydy started.Can  still enroll it ?"

In [28]:
results = index.search(query=q,num_results=10,filter_dict={'course':'data-engineering-zoomcamp'})

In [16]:
client = OpenAI()

In [20]:
response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [{'role':'user','content':q}]
)

In [24]:
print(response.choices[0].message.content)

It's possible that you can still enroll in a course that has already started, but it depends on several factors. Here are steps you can take to find out:

1. **Check the Course Details**: Look for information on the course's webpage or the academic institution's website. Some courses allow late enrollment within a certain period after the start date.

2. **Contact the Instructor or Course Coordinator**: Reach out to the person teaching or managing the course. They can give you a definitive answer and may also provide additional instructions for catching up on missed material.

3. **Check Enrollment Policies**: Some institutions have specific policies regarding late enrollments. Review these policies which are usually available on the institution's academic affairs or registrar's office website.

4. **Consider Online Platforms**: If the course is offered through an online learning platform, check their support or help section for guidelines on late enrollments.

5. **Registration Office

In [34]:
prompt_template = """
You are course a teaching assistant. Answer the QUESTION based on CONTEXT from FAQ database.
Use only facts from the context when answering the question.
If CONTEXT doesn't contain the answer output is none
QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [31]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [35]:
prompt = prompt_template.format(question=q,context=context).strip()

In [40]:
response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [{'role':'user','content':prompt}]
)
print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
